In [1]:
# For importing converted final tsv into sqlite
import os
import csv
csv.field_size_limit(100000000)
import time
import sqlite3
conn = sqlite3.connect('../db.sqlite3')
conn.isolation_level=None
cur = conn.cursor()
cur.execute("PRAGMA journal_mode=WAL")
directory_name = os.path.join(os.pardir, 'coreapi/dataset/imdb/')
buffer_size = 100000

In [2]:
clean = lambda val: val.strip() if val not in ["\\N", ""] else None
namebasics = lambda i,row: (row[0], row[1])
titleakas = lambda i,row: (i, row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[0])
titlebasics = lambda i,row: (*row, None)
titleprincipals = lambda i,row: (i, row[1], row[3], row[4], row[5], row[2], row[0])
titlerating = lambda i,row: (i, row[1], row[2], row[0])
titlelink = lambda i,row: (i, *row)

writer_table   = "coreapi_titlebasics_writers" 
director_table = "coreapi_titlebasics_directors"
crew_file = "title_crew.tsv"

def flush_buffer(table_name, buffer, n_col):
    q_m = ("?, "*n_col)[:-2]
    sql= "insert or replace into " + table_name + " values("+ q_m + ")"
    cur.executemany(sql, buffer)
    buffer.clear()

def insert_table(table_name, file_name, f, n_col, sep="\t"):
    read_file = os.path.join(directory_name, file_name)
    reader = csv.reader(open(read_file), delimiter=sep)
    start_time = time.time()
    col_s = len(next(reader))
    try:
        cur.execute("begin")
        buffer=[]
        for idx, row in enumerate(reader,1):
            if len(row) == col_s:
                row = map(clean, row)
                row = f(idx, row)
                buffer.append(row)
                if not idx%buffer_size:
                    flush_buffer(table_name, buffer, n_col)
                    print("\rwritten", f'{idx:,}', "entries of ", table_name, end="")

        if len(buffer):
            flush_buffer(table_name, buffer, n_col)

        cur.execute("commit")
        print("\nTotal time ", time.time()-start_time)
    except Exception as e:
        print(e)
        cur.execute("rollback")


In [ ]:
def insert_crew():
    start_time = time.time()
    with open(read_file) as r:
        reader = csv.reader(r, delimiter="\t")
        column_names = next(reader)
        ir=0
        iw=0
        writer_buffer=[]
        director_biffer=[]
        for cnt, row in enumerate(reader, 1):
            tconst = is_valid(row[0])
            directors = [clean(i) for i in row[1].split(',')]
            directors = row[1].split(',')
            writers = row[2].split(',')
            for d in directors:
                director_buffer.append(map(clean, [ir, tconst, d]))
                ir+=1
            for w in writers:
                writer_buffer.append((map(clean, [ir, tconst, w])))
                iw+=1
            if not cnt%buffer_size:
                flush_buffer(writer_table, writer_buffer, 3)
                flush_buffer(director_table, director_buffer, 3)
                print("written", f'{iw:,}', "writers ", f'{ir:,}', "directors", end="\r")

    print("Converted ", table_name, " in ", time.time()-start_time)

In [3]:
tables = [
#     ("coreapi_namebasics", "name_basics.tsv", namebasics, 2),
    ("coreapi_titlebasics", "title_basics.tsv", titlebasics, 10),
#     ("coreapi_titleakas", "title_akas.tsv", titleakas, 9)
#     ("coreapi_link", "links_final.tsv", 3),
#     ("coreapi_titlerating", "title_ratings.tsv", 4),
#     ("coreapi_titleprincipals", "title_principals.tsv", 7)
         ]
for table in tables:
    insert_table(*table)
    cur.execute("PRAGMA wal_checkpoint")
# insert_crew()
cur.execute("PRAGMA wal_checkpoint")
conn.close()

written 10,800,000 entries of  coreapi_titlebasics
Total time  311.96034026145935
